In [1]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time


In [2]:
PATH_TO_DATA = '../input/'

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'train_features.csv'), 
                                    index_col='match_id_hash')
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            'train_targets.csv'), 
                                   index_col='match_id_hash')


In [3]:
df_train_features.head(n=2)

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,r1_max_mana,r1_level,r1_x,r1_y,r1_stuns,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_obs_placed,r1_sen_placed,r2_hero_id,r2_kills,r2_deaths,r2_assists,r2_denies,r2_gold,r2_lh,r2_xp,r2_health,r2_max_health,r2_max_mana,...,d4_health,d4_max_health,d4_max_mana,d4_level,d4_x,d4_y,d4_stuns,d4_creeps_stacked,d4_camps_stacked,d4_rune_pickups,d4_firstblood_claimed,d4_teamfight_participation,d4_towers_killed,d4_roshans_killed,d4_obs_placed,d4_sen_placed,d5_hero_id,d5_kills,d5_deaths,d5_assists,d5_denies,d5_gold,d5_lh,d5_xp,d5_health,d5_max_health,d5_max_mana,d5_level,d5_x,d5_y,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,543,7,533,358,600,350.93784,2,116,122,0.0,0,0,1,0,0.0000,0,0,0,0,78,0,0,0,3,399,4,478,636,720,254.93774,...,760,760,326.9378,2,90,150,0.0,0,0,2,1,1.0,0,0,1,0,34,0,0,0,0,851,11,870,593,680,566.93805,3,128,128,0.0,0,0,0,0,0.0,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,5257,52,3937,1160,1160,566.93805,8,76,78,0.0,0,0,0,0,0.4375,0,0,0,0,96,3,1,2,3,3394,19,3897,1352,1380,386.93787,...,567,1160,410.9379,6,124,142,0.0,0,0,6,0,0.5,0,0,0,0,92,0,2,0,1,1423,8,1136,800,800,446.93793,4,180,176,0.0,0,0,0,0,0.0,0,0,0,0


**We have nearly 40K examples with each described with match_id_hash and 245 features.**

In [4]:
df_train_features.shape

(39675, 245)

In [5]:
y_target=df_train_targets['radiant_win'].apply(lambda x:int(x)).values

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train_features.values, y_target, 
                                                      test_size=0.3, 
                                                      random_state=17)

**Training a simple Catboost classifier**

In [7]:
%%time
base_model = CatBoostClassifier(task_type = "GPU",verbose=True)
base_model.fit(X_train,y_train,)

Learning rate set to 0.042921
0:	learn: 0.6861814	total: 64.4ms	remaining: 1m 4s
1:	learn: 0.6797877	total: 126ms	remaining: 1m 3s
2:	learn: 0.6744033	total: 188ms	remaining: 1m 2s
3:	learn: 0.6687509	total: 249ms	remaining: 1m 2s
4:	learn: 0.6637175	total: 307ms	remaining: 1m 1s
5:	learn: 0.6587835	total: 365ms	remaining: 1m
6:	learn: 0.6542319	total: 425ms	remaining: 1m
7:	learn: 0.6501581	total: 482ms	remaining: 59.8s
8:	learn: 0.6465399	total: 539ms	remaining: 59.4s
9:	learn: 0.6430284	total: 597ms	remaining: 59.1s
10:	learn: 0.6392932	total: 654ms	remaining: 58.8s
11:	learn: 0.6357444	total: 716ms	remaining: 59s
12:	learn: 0.6324435	total: 774ms	remaining: 58.8s
13:	learn: 0.6293696	total: 832ms	remaining: 58.6s
14:	learn: 0.6262487	total: 889ms	remaining: 58.4s
15:	learn: 0.6236081	total: 946ms	remaining: 58.2s
16:	learn: 0.6217114	total: 1s	remaining: 57.9s
17:	learn: 0.6194533	total: 1.06s	remaining: 57.8s
18:	learn: 0.6168013	total: 1.12s	remaining: 57.6s
19:	learn: 0.6148038	

Making predictions on holdout set

In [8]:
y_pred = base_model.predict_proba(X_valid)[:, 1]
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.8026054819101719


Preparing a submission file

In [9]:
def submit_predictions(name,model):
    df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_features.csv'), 
                                   index_col='match_id_hash')
    X_test = df_test_features.values
    y_test_pred = model.predict_proba(X_test)[:, 1]

    df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=df_test_features.index)
    submission_filename = 'submission_{}.csv'.format(name)
    df_submission.to_csv(submission_filename)
    print('Submission saved to {}'.format(submission_filename))

In [10]:
submit_predictions('simple_cat_boost',model=base_model)

Submission saved to submission_simple_cat_boost.csv


Now, using Bayesian optimization to find optimal parameters.
Code has been taken from https://github.com/lmassaron/kaggledays-2019-gbdt/blob/master/Kaggle%20Days%20Paris%20-%20Skopt%20%2B%20CatBoost%20solution.ipynb

In [11]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [12]:
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [13]:
clf = CatBoostClassifier(thread_count=2,
                         loss_function='Logloss',
                        
                         od_type = 'Iter',
                         verbose= False
                        )

In [14]:
# Defining your search space
search_spaces = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 'scale_pos_weight':Real(0.01, 1.0, 'uniform')}


In [15]:
# Setting up BayesSearchCV
opt = BayesSearchCV(clf,
                    search_spaces,
                    scoring=roc_auc,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)

In [16]:
# best_params = report_perf(opt, df_train_features, y_target,'CatBoost', 
#                           callbacks=[VerboseCallback(100), 
#                                      DeadlineStopper(60*10)])

In [17]:
best_params={'bagging_temperature': 0.41010395885331385,
 'border_count': 186,
 'depth': 8,
 'iterations': 323,
 'l2_leaf_reg': 21,
 'learning_rate': 0.0673344419215237,
 'random_strength': 3.230824361824754e-06,
 'scale_pos_weight': 0.7421091918485163}

In [18]:
best_params['iterations']=1000

Making a classifer based on tuned paramters

In [19]:
# %%time
# tuned_model = CatBoostClassifier(**best_params,task_type = "GPU",od_type='Iter',one_hot_max_size=10)
# tuned_model.fit(X_train,y_train)

In [20]:
# %%time
# y_pred = tuned_model.predict_proba(X_valid)[:, 1]
# valid_score = roc_auc_score(y_valid, y_pred)
# print('Validation ROC-AUC score:', valid_score)
# ##ROC_AUC is 0.8056

In [21]:
%%time
tuned_model = CatBoostClassifier(**best_params,task_type = "GPU",od_type='Iter',one_hot_max_size=10)
tuned_model.fit(df_train_features,y_target)

0:	learn: 0.6808245	total: 203ms	remaining: 3m 22s
1:	learn: 0.6697772	total: 393ms	remaining: 3m 16s
2:	learn: 0.6601326	total: 583ms	remaining: 3m 13s
3:	learn: 0.6507361	total: 771ms	remaining: 3m 12s
4:	learn: 0.6424479	total: 959ms	remaining: 3m 10s
5:	learn: 0.6348811	total: 1.15s	remaining: 3m 10s
6:	learn: 0.6284094	total: 1.34s	remaining: 3m 9s
7:	learn: 0.6224662	total: 1.53s	remaining: 3m 9s
8:	learn: 0.6168351	total: 1.72s	remaining: 3m 8s
9:	learn: 0.6120097	total: 1.9s	remaining: 3m 8s
10:	learn: 0.6080169	total: 2.08s	remaining: 3m 7s
11:	learn: 0.6039145	total: 2.27s	remaining: 3m 7s
12:	learn: 0.5999633	total: 2.46s	remaining: 3m 6s
13:	learn: 0.5961684	total: 2.65s	remaining: 3m 6s
14:	learn: 0.5927855	total: 2.83s	remaining: 3m 5s
15:	learn: 0.5896720	total: 3.02s	remaining: 3m 5s
16:	learn: 0.5867943	total: 3.2s	remaining: 3m 5s
17:	learn: 0.5842251	total: 3.4s	remaining: 3m 5s
18:	learn: 0.5816089	total: 3.58s	remaining: 3m 5s
19:	learn: 0.5793047	total: 3.77s	rema

In [22]:
submit_predictions('tuned_cat_boost',model=tuned_model)

Submission saved to submission_tuned_cat_boost.csv
